In [2]:
# Biblioteki wejściowe
import pandas as pd
import numpy as np
import posixpath as os
import pathlib
from pathlib import Path
import pyodbc
import sys
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.tseries.offsets import MonthEnd
import IPython
import math

pd.set_option("display.max_rows", None)
#pd.options.display.max_rows=50000
pd.options.display.max_colwidth=70
#pd.options.display.min_colwidth=10
pd.options.display.precision=2
pd.options.display.max_columns=50
pd.options.display.float_format='{0:,.2f}'.format

DATA = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/FINANSE (RAPORTY)/STRATEGIA 2023-2028/DATA')
ASSETS = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/FINANSE (RAPORTY)/STRATEGIA 2023-2028/ASSETS')
IMAGES = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/FINANSE (RAPORTY)/STRATEGIA 2023-2028/IMAGES')

In [3]:
# ZAPYTANIE SQL - Środki Trwałe

t_sql="SELECT * FROM SrodkiTrwale"

cnxn=pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                    'Server=192.168.1.252\ENOVA_KOPALNIA;'
                    'Database=OGORZELEC;'
                    'UID=ro_daniel.wojcik;'
                    'PWD=661594050xkr@;'
                    'Trusted_Connection=no;')

t_zapisy_ST=pd.read_sql_query (t_sql,cnxn)
t_zapisy_ST.to_csv(DATA /'ST_Zapisy_SQL.csv',index=False)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [47]:
# ZAPYTANIE SQL - Środki Trwałe

t_sql="SELECT Typ, Kierunek, Data, SrodekTrwaly, WartoscBilansowaValue FROM ObrotyST;"

cnxn=pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                    'Server=192.168.1.252\ENOVA_KOPALNIA;'
                    'Database=OGORZELEC;'
                    'UID=ro_daniel.wojcik;'
                    'PWD=661594050xkr@;'
                    'Trusted_Connection=no;')

t_zapisy_OST=pd.read_sql_query (t_sql,cnxn)
t_zapisy_OST.to_csv(DATA /'OST_Zapisy_SQL.csv',index=False)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [39]:
# Pobiera zapisy z pliku

t_zapisy_ST=pd.read_csv(DATA /'ST_Zapisy_SQL.csv')
t_zapisy_OST=pd.read_csv(DATA /'OST_Zapisy_SQL.csv')
t_zapisy_ST['BO']=t_zapisy_ST['WartoscPoczatkowaBilansowaBOValue']-t_zapisy_ST['OdpisyBilansoweBOValue']
t_ST=t_zapisy_ST[['ID', 'NumerInwentarzowy', 'Nazwa','BO']]
t_zapisy_OST['Obroty']=t_zapisy_OST.apply(lambda x:x['WartoscBilansowaValue'] if x['Kierunek']==1 else -x['WartoscBilansowaValue'],axis=1)

### Raport środków trwałych

In [40]:
t_ST_Odpisy=t_zapisy_OST.groupby(['SrodekTrwaly'],as_index=False)[['Obroty']].sum()
t_ST=pd.merge(t_ST,t_ST_Odpisy,how='inner',left_on=['ID'],right_on=['SrodekTrwaly'])
t_ST['WNA']=t_ST['BO']+t_ST['Obroty']
t_ST=t_ST[t_ST['WNA']>1]
t_ST=t_ST[['ID', 'NumerInwentarzowy', 'Nazwa','WNA']]
t_ST_Data_Max=t_zapisy_OST.groupby(['SrodekTrwaly'],as_index=False)[['Data']].max()
t_ST_Amortyzacja=t_zapisy_OST[t_zapisy_OST['Typ']==3]
t_ST_Amortyzacja=t_ST_Amortyzacja[['SrodekTrwaly','Data','Obroty']]
t_ST_Amortyzacja=pd.merge(t_ST_Data_Max,t_ST_Amortyzacja,how='inner',on=['SrodekTrwaly','Data'])
t_ST_Amortyzacja=t_ST_Amortyzacja[t_ST_Amortyzacja['Data']=='2022-09-30']
t_ST_Amortyzacja.rename(columns={'SrodekTrwaly':'ID'},inplace=True)
t_ST=pd.merge(t_ST,t_ST_Amortyzacja,how='left',on='ID')
t_ST.rename(columns={'NumerInwentarzowy':'Numer ST', 'Nazwa':'Nazwa ST', 'WNA':'Wartość Bilansowa',  'Obroty':'Amortyzacja'},inplace=True)

t_ST['Data']=t_ST.apply(lambda x:pd.to_datetime('2022-09-30') if x['ID']==306 else x['Data'],axis=1)
t_ST['Amortyzacja']=t_ST.apply(lambda x:-x['Wartość Bilansowa']/60 if x['ID']==306 else x['Amortyzacja'],axis=1)

In [52]:
t_ST.sum()

/var/folders/7l/tjnfp_9n18j7n_6s3mbhg19h0000gn/T/ipykernel_987/661818759.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  t_ST.sum()


ID                                                                                   25825
Numer ST             001/02101/121104/124104/191101/122101/126109/125109/167104/118105/...
Nazwa ST             WU-WIEŚCISZOWICEBUDYNEK WARSZTATU MECHANICZNEGOBUDYNEK KOTŁOWNI-MA...
Wartość Bilansowa                                                            11,344,943.80
Amortyzacja                                                                    -129,681.85
dtype: object

In [48]:
t_ST_Plan=pd.DataFrame()

for index,row in t_ST.iterrows():
    NUMER=row['Numer ST']
    NAZWA=row['Nazwa ST']
    WNA=row['Wartość Bilansowa']
    START=pd.to_datetime('2022-09-30')
    AM=row['Amortyzacja']

    OKRES=math.floor(abs(WNA/AM))
    RESZTA=WNA+OKRES*AM

    temp_okres=pd.date_range(start=START,freq='M',periods=OKRES+1)
    t_ST_temp=pd.DataFrame({'DATE':temp_okres})
    KONIEC=t_ST_temp['DATE'].max()

    t_ST_temp['Numer ST']=NUMER
    t_ST_temp['Nazwa ST']=NAZWA
    t_ST_temp['BO']=t_ST_temp.apply(lambda x:WNA if x['DATE']==START else 0,axis=1)
    t_ST_temp['Amortyzacja']=t_ST_temp.apply(lambda x:AM if x['DATE']>START else 0,axis=1)
    t_ST_temp['Amortyzacja']=t_ST_temp.apply(lambda x:AM-RESZTA if x['DATE']==KONIEC else x['Amortyzacja'],axis=1)
    t_ST_temp['BZ']=t_ST_temp['BO']+t_ST_temp['Amortyzacja']
    t_ST_temp['BZ']=t_ST_temp['BZ'].cumsum()


    t_ST_Plan=pd.concat([t_ST_Plan,t_ST_temp],ignore_index=True)

In [53]:
t_ST_Plan.sum()

/var/folders/7l/tjnfp_9n18j7n_6s3mbhg19h0000gn/T/ipykernel_987/439486127.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  t_ST_Plan.sum()


Numer ST       001/02001/02001/02001/02001/02001/02001/02001/02001/02001/02001/02...
Nazwa ST       WU-WIEŚCISZOWICEWU-WIEŚCISZOWICEWU-WIEŚCISZOWICEWU-WIEŚCISZOWICEWU...
BO                                                                     11,344,943.80
Amortyzacja                                                           -11,344,943.80
BZ                                                                    809,245,843.01
dtype: object